In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
from pymongo import MongoClient


def weather_fun():
    response = urlopen("https://tw.news.yahoo.com/weather/%E8%87%BA%E7%81%A3/%E6%A1%83%E5%9C%92%E5%B8%82/%E6%A1%83%E5%9C%92%E5%B8%82-2298866")
    html = BeautifulSoup(response, "html5lib")
    weathers = html.select('div.BdB')

    today = datetime.date.today()
    weather = {}
    for i in range(10):
        day = weathers[i].find('span').text
        day_weather = weathers[i].find_all('img')[0]['alt']
        rain = weathers[i].find_all('img')[1]['alt']
        high_t = weathers[i].find('span', class_='high').text
        low_t = weathers[i].find('span', class_='low').text
        result = str(today + datetime.timedelta(days=i))
        m = result[5:7]
        d = result[8:10]
        weather[str(i)] = m + '/' + d + '*' + day + '*' + day_weather + '*' + rain + '*' + high_t + '*' + low_t

    week_day = []
    for i in range(10):
        week_day.append(weathers[i].find('span').text)

    return weather


def insert_mongodb(weather_forecast):
    conn = MongoClient('mongo')
    db = conn.test
    collection = db.weather

    m = str(datetime.date.today())[5:7]
    d = str(datetime.date.today())[8:]
    now = m + '/' + str(int(d))
    collection.insert({'day': now, 'weather': weather_forecast})


if __name__ == '__main__':
    weather = weather_fun()
    insert_mongodb(weather)
    print(weather)

{'0': '05/26*星期日*局部雷陣雨*降雨機率: 15%*33°*24°', '1': '05/27*星期一*雷雨*降雨機率: 60%*31°*25°', '2': '05/28*星期二*局部雷陣雨*降雨機率: 60%*27°*23°', '3': '05/29*星期三*多雲*降雨機率: 0%*26°*22°', '4': '05/30*星期四*局部多雲*降雨機率: 10%*29°*22°', '5': '05/31*星期五*雷雨*降雨機率: 10%*30°*24°', '6': '06/01*星期六*雷雨*降雨機率: 45%*29°*26°', '7': '06/02*星期日*雷雨*降雨機率: 0%*30°*26°', '8': '06/03*星期一*雷雨*降雨機率: 60%*29°*26°', '9': '06/04*星期二*雷雨*降雨機率: 45%*31°*26°'}


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
